**START OF THE PREPROCESS**

In [1]:
"""
PREPROCESS
1. RESIZE THE INPUT
2. ADJUST THE SKIN COLOR USING THE OVERALL AVG OF ALL DATASET INPUTS
3. DETECT EDGE OF OBJECTS IN THE IMAGE TO STRENGTHEN THE EDGE AND DENOISE
4. USE PROCESSED IMAGE TO DETECT FINGERS/ HANDS (NOT WRIST)
5. DATA AUGMENTATION (OPTIONAL)
"""

import os
import numpy as np
import matplotlib.pyplot as plt
import random
import shutil
import math

import torch
import torchvision
import cv2
import mediapipe as mp

from PIL import Image
from PIL import ImageEnhance
from torchvision.transforms import functional as F
from cvzone.HandTrackingModule import HandDetector

**START PROCESS IMAGES**

In [ ]:
"""
STEP 1 - RESIZE
"""

def resize_image(input_image, target_size=(224, 224)):
    # Resize the image
    resized_image = input_image.resize(target_size)
    
    return resized_image

In [ ]:
"""
STEP 2 - ADJUST LIGHTING
"""

def adjust_lighting(image, avg_skin_brightness):
    img = np.array(image)
    # Convert to HSV and create a skin mask
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    lower_skin, upper_skin = np.array([0, 20, 70]), np.array([20, 255, 255])
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    skin_pixels = hsv[:, :, 2][mask > 0]
    
    if len(skin_pixels) > 0:
        avg_skin_brightness_img = np.mean(skin_pixels)
        target_brightness = avg_skin_brightness  # Use global average brightness
        factor = target_brightness / avg_skin_brightness_img
        hsv[:, :, 2] = np.where(mask > 0, np.clip(hsv[:, :, 2] * factor, 0, 255), hsv[:, :, 2])
    
    img_adjusted = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    return Image.fromarray(img_adjusted)

In [ ]:
"""
STEP 3 - SHARPEN IMAGE
"""

def adjust_contrast(image, factor=1.2):
    enhancer = ImageEnhance.Contrast(image)
    enhanced_image = enhancer.enhance(factor)
    
    return enhanced_image

def super_resolution(image, scale_factor=2):
    image = np.array(image)
    
    # Use cubic interpolation for resizing the image
    height, width = image.shape[:2]
    new_dimensions = (int(width * scale_factor), int(height * scale_factor))
    
    resized_image = cv2.resize(image, new_dimensions, interpolation=cv2.INTER_CUBIC)
    
    return Image.fromarray(resized_image)


In [ ]:
"""
STEP 4 - DETECT HANDS UPDATED
"""

detector = HandDetector(maxHands=2)
all_landmarks = {}

def detect_and_plot_hands_updated(image, image_name):

    image_cv = np.array(image)
    image_cv = cv2.cvtColor(image_cv, cv2.COLOR_RGB2BGR)

    # Find hands in the image (with landmarks drawn on the image)
    hands, image_with_hands = detector.findHands(image_cv, draw=False)

    black_bg_image = np.zeros_like(image_cv)

    # Draw landmarks on the black background if hands are detected
    if hands:
        for hand in hands:
            for lm in hand['lmList']:
                # Draw red dot for each landmark on the black background
                cv2.circle(black_bg_image, (lm[0], lm[1]), 5, (0, 0, 255), -1)  # Red color
                cv2.circle(image_with_hands, (lm[0], lm[1]), 5, (0, 0, 255), -1)  # Red color
        

    image_with_hands_rgb = cv2.cvtColor(image_with_hands, cv2.COLOR_BGR2RGB)
    black_bg_image_rgb = cv2.cvtColor(black_bg_image, cv2.COLOR_BGR2RGB)

    # Save the images
    image_with_hands_pil = Image.fromarray(image_with_hands_rgb)
    black_bg_image_pil = Image.fromarray(black_bg_image_rgb)

    # Return the number of hands detected and the two output images
    num_hands = len(hands)
    print(f"Number of hands detected in {image_name}: {num_hands}")

    return num_hands, image_with_hands_pil, black_bg_image_pil

In [ ]:
"""
INPUTS FOR THIRD TRIAL W/ HAND DETECTOR
"""

input_folder = "TSL_Sonner_100_selected" 
output_folder = "TSL_Sonner_100_selected_combined_preprocessed"
output_folder_detected = "TSL_Sonner_100_selected_combined_detected"
output_folder_detected_mp = "TSL_Sonner_100_selected_combined_detected_mp"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_detected, exist_ok=True)
os.makedirs(output_folder_detected_mp, exist_ok=True)

# Initialize variables
one_hand_count = 0
two_hands_count = 0
total_images = 0
class_hand_count = {}

In [ ]:
"""
RUN ALL W/ HAND DETECTOR
"""
class_index_counter = {}

for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
    
        input_path = os.path.join(input_folder, filename)
        image = Image.open(input_path).convert("RGB")
        total_images += 1
    
        # STEP 1
        image_resized = resize_image(image)
        
        # STEP 2-2
        adjusted_image = adjust_lighting(image_resized, target_brightness)
    
        # STEP 3
        sharpen_image = super_resolution(adjust_contrast(adjusted_image))
        
        # Save the processed image
        output_path = os.path.join(output_folder, f"{filename}")
        sharpen_image.save(output_path)
    
        # Step 4: Detect hands and process image
        num_hands, finger_detected_img, finger_detected_mp_img = detect_and_plot_hands_updated(image, filename)
        
        class_name = filename.split(' ')[0]  # Assuming class is determined by the first part of the filename
        if class_name not in class_hand_count:
            class_hand_count[class_name] = {'1_hand': 0, '2_hands': 0, 'no_hand': 0}

        # Initialize the index counter for each class if not already set
        if class_name not in class_index_counter:
            class_index_counter[class_name] = 0  # Change the index number based on usage

        # Generate filenames with the format '(classname) (index).jpg'
        index = class_index_counter[class_name]
        class_index_counter[class_name] += 1
        
        output_filename_detected = f"{class_name} ({index}).jpg"
        output_filename_detected_mp = f"{class_name} ({index}).jpg"

        output_folder_detected_path = os.path.join(output_folder_detected, output_filename_detected)
        output_folder_detected_mp_path = os.path.join(output_folder_detected_mp, output_filename_detected_mp)

        # Count number of hands and save detected hands img + mp img
        if num_hands == 1:
            class_hand_count[class_name]['1_hand'] += 1
        
        elif num_hands == 2:
            class_hand_count[class_name]['2_hands'] += 1
            
        else:
            class_hand_count[class_name]['no_hand'] += 1

        finger_detected_img.save(output_folder_detected_path)
        finger_detected_mp_img.save(output_folder_detected_mp_path)
    
        # Display input and output images
        print(f"{total_images}")
        plt.figure(figsize=(9, 3))
        plt.subplot(1, 3, 1)
        plt.title("INPUT")
        plt.imshow(image)
        plt.axis("off")
        
        plt.subplot(1, 3, 2)
        plt.title("OUTPUT IMAGE")
        plt.imshow(finger_detected_img)
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.title("OUPUT IMAGE MP")
        plt.imshow(finger_detected_mp_img)
        plt.axis("off")
        
        plt.show()

# Print the counts per class
print(f"Total Images Processed: {total_images}")
for class_name, counts in class_hand_count.items():
    print(f"Class: {class_name}")
    print(f"  Images with 1 Hand Detected: {counts['1_hand']}")
    print(f"  Images with 2 Hands Detected: {counts['2_hands']}")
    print(f"  Images with No Hands Detected: {counts['no_hand']}")
    print() 